In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.storage.table import TableAdapter

In [2]:
table_name = "training"

table_adapter: TableAdapter = TableAdapter()

data_path = "/data/parquet/"

parquet_process_data_path = data_path + "processed_data.parquet"

In [3]:
print(f"Reading from parquet {parquet_process_data_path} with Updated Primary Captions")

processed_with_captions = pandas.read_parquet(parquet_process_data_path)

display(processed_with_captions)

Reading from parquet /data/parquet/processed_data.parquet with Updated Primary Captions


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...,a woman holding up a bottle of beer
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...,a young woman in a red dress is talking on the...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...,a bridge over a body of water with a waterfall
8596,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,a series of photos showing a man in a suit


In [4]:
from typing import List
from dataclasses import dataclass

@dataclass
class DataSources:
	name: str
	data: List[str]

	@staticmethod
	def from_dict(obj: dict) -> 'DataSources':
		_name = obj.get("name")
		_data = [x for x in obj.get("data")]
		return DataSources(_name, _data)

In [5]:
sources = [
	{"name": "CityDiffusion", "data": [
		"CityPorn"
	]},
	{"name": "NatureDiffusion",
	 "data": [
		 "EarthPorn"
	 ]},
	{"name": "CosmicDiffusion",
	 "data": [
		 "SpacePorn"
	 ]},
	{"name": "MemeDiffusion",
	 "data": [
		 "greentext",
		 "memes"
	 ]},
	{"name": "SexyDiffusion",
	 "data": [
		 "sfwpetite",
		 "selfies",
		 "Amicute",
		 "amihot",
		 "AmIhotAF",
		 "HotGirlNextDoor",
		 "SFWNextDoorGirls",
		 "SFWRedheads"]
	 }
]

data_sources = [DataSources.from_dict(x) for x in sources]

In [7]:
lines = []
for record in processed_with_captions.to_dict(orient='records'):
	subreddit = record['subreddit']
	for source in data_sources:
		if subreddit in source.data:
			line = f"<|startoftext|><|model|>{source.name}<|prompt|>{record['text']}<|text|>{record['original_caption']}<|endoftext|>" + "\n"
			lines.append(line)
with open("training.txt", "wb") as f:
	for line in lines:
		f.write(line.encode("utf-8"))